### Instalaciones

In [ ]:
# Instalación de Librerías (ejecutar solo una vez si no están instaladas)
# !pip install langgraph langchain langchain_aws boto3 python-dotenv anthropic # Añadimos anthropic por si usamos Claude directamente


### Importar librerias

In [ ]:
import os
import boto3
import json
from typing import TypedDict, Annotated, List, Union
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage 
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.sqlite import SqliteSaver 
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate


# from dotenv import load_dotenv
# load_dotenv() 


### Configuración del Cliente de Bedrock y LLM

In [ ]:

try:
    
    bedrock_runtime_client = boto3.client(
        service_name="bedrock-runtime",
        # region_name=region_name 
    )
    print("Cliente de Bedrock Runtime inicializado exitosamente.")
except Exception as e:
    print(f"Error inicializando el cliente de Bedrock Runtime: {e}")
    print("Por favor, verifica tu configuración de credenciales y región de AWS.")
    


MODEL_ID_HAIKU = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID_SONNET = "anthropic.claude-3-sonnet-20240229-v1:0" 


try:
    llm = ChatBedrock(
        client=bedrock_runtime_client, 
        model_id=MODEL_ID_HAIKU,
        model_kwargs={
            "temperature": 0.1, 
            "max_tokens": 1024    
        }
    )
    print(f"LLM ({MODEL_ID_HAIKU}) inicializado exitosamente con ChatBedrock.")
except Exception as e:
    print(f"Error inicializando ChatBedrock con {MODEL_ID_HAIKU}: {e}")
    print("Asegúrate de que el modelo está habilitado en tu cuenta de Bedrock para la región configurada.")


### Definición del Estado del Grafo (AgentState)

In [ ]:

class AgentState(TypedDict):
    userInput: str
    intent: str
    entities: dict
    catalogQueryResult: List[dict]
    finalResponse: str
    callLog: List[str] 
    # (Opcional para Nivel 1 ) supervisor_notes: str
    # (Opcional para Nivel 1 ) error_message: str 


### Definición del Nodo - Interpretar Entrada del Usuario (NLU)

In [ ]:
def interpret_user_input(state: AgentState):
    """
    Toma la entrada del usuario y usa el LLM para extraer la intención y las entidades.
    """
    print("---NODO: Interpretando Entrada del Usuario---")
    user_input = state["userInput"]
    current_call_log = state.get("callLog", [])


    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(
            content=(
                "Eres un asistente experto en procesamiento de lenguaje natural para un sistema de compras. "
                "Tu tarea es analizar la consulta del usuario y extraer su intención principal y las entidades relevantes. "
                "Las intenciones pueden ser: 'buscar_producto', 'comparar_productos', 'pedir_recomendacion', 'ver_carrito', 'saludar', 'despedirse', 'otra'. "
                "Las entidades comunes son: 'nombre_producto', 'marca', 'categoria', 'color', 'talla', 'precio_maximo', 'caracteristicas_adicionales'. "
                "Responde ÚNICAMENTE con un objeto JSON que contenga dos claves: 'intent' y 'entities'. "
                "La clave 'entities' debe ser un diccionario de las entidades encontradas. Si no encuentras una entidad, no la incluyas. "
                "Si no estás seguro de la intención o no puedes extraer entidades útiles, puedes usar la intención 'otra' y un diccionario de entidades vacío."
                "Ejemplo de salida: {\"intent\": \"buscar_producto\", \"entities\": {\"categoria\": \"televisor\", \"marca\": \"LG\", \"tamaño\": \"55 pulgadas\"}}"
            )
        ),
        HumanMessage(content=f"Analiza la siguiente consulta del usuario: {user_input}")
    ])

    try:
        formatted_prompt = prompt_template.format_messages(userInput=user_input)
        ai_response = llm.invoke(formatted_prompt) 

        response_content = ai_response.content
        print(f"Respuesta cruda del LLM para NLU: {response_content}")

        parsed_response = json.loads(response_content)
        intent = parsed_response.get("intent", "otra")
        entities = parsed_response.get("entities", {})

        current_call_log.append(f"NLU: Input='{user_input}', Intent='{intent}', Entities='{json.dumps(entities)}', LLM_Raw_Output='{response_content[:200]}...'") 
        print(f"Intención extraída: {intent}")
        print(f"Entidades extraídas: {entities}")

        return {
            "intent": intent,
            "entities": entities,
            "callLog": current_call_log
        }

    except json.JSONDecodeError as e:
        print(f"Error al decodificar la respuesta JSON del LLM: {e}")
        print(f"Respuesta del LLM que causó el error: {response_content}")
        current_call_log.append(f"NLU_ERROR: Error decodificando JSON. LLM_Raw_Output='{response_content[:200]}...'")
        return {
            "intent": "error_nlu",
            "entities": {},
            "callLog": current_call_log
        }
    except Exception as e:
        print(f"Error inesperado durante la NLU: {e}")
        current_call_log.append(f"NLU_ERROR: Error inesperado - {str(e)}")
        return {
            "intent": "error_nlu",
            "entities": {},
            "callLog": current_call_log
        }

### Definición del Nodo - Consultar Catálogo de Productos

In [ ]:
import json 
import os   

def load_product_database(db_path="../data/products.json"):
    """
    Carga la base de datos de productos desde un archivo JSON.
    Asume que el archivo products.json está en una carpeta 'data'
    ubicada un nivel arriba de la carpeta donde se encuentra este notebook.
    """
   
    # Para verificar la ruta actual desde donde se ejecuta el notebook 
    # print(f"Directorio de trabajo actual: {os.getcwd()}")
    # print(f"Intentando cargar DB desde: {os.path.abspath(db_path)}")

    try:
        with open(db_path, 'r', encoding='utf-8') as f:
            database = json.load(f)
        print(f"Base de datos de productos cargada exitosamente desde '{os.path.abspath(db_path)}'")
        return database
    except FileNotFoundError:
        print(f"ERROR CRÍTICO: Archivo de base de datos no encontrado en '{os.path.abspath(db_path)}'.")
        print("Asegúrate de que la carpeta 'data' exista en la raíz de tu proyecto y contenga 'products.json'.")
        print(f"Directorio actual: {os.getcwd()}")
        return []
    except json.JSONDecodeError:
        print(f"ERROR CRÍTICO: Error al decodificar el archivo JSON en '{os.path.abspath(db_path)}'. Verifica el formato del archivo.")
        return []
    except Exception as e:
        print(f"ERROR CRÍTICO: Ocurrió un error inesperado al cargar la base de datos: {e}")
        return []

def query_product_catalog(state: AgentState):
    """
    Consulta un catálogo de productos (cargado desde un archivo JSON) basado en las entidades extraídas.
    """
    print("---NODO: Consultando Catálogo de Productos (desde Archivo JSON)---")
    entities = state.get("entities", {})
    intent = state.get("intent", "")
    current_call_log = state.get("callLog", [])

    
    product_database = load_product_database() 

    if not product_database: 
        current_call_log.append("CATALOG_ERROR: La base de datos de productos no se pudo cargar o está vacía.")
        print("Error: La base de datos de productos está vacía o no se pudo cargar. Revisa los mensajes anteriores.")
        return {"catalogQueryResult": [], "callLog": current_call_log}

    
    if intent not in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
        print(f"Intención '{intent}' no requiere consulta de catálogo. Omitiendo.")
        current_call_log.append(f"CATALOG_SKIP: Intención '{intent}' no requiere consulta.")
        return {"catalogQueryResult": [], "callLog": current_call_log}

    print(f"Consultando catálogo con entidades: {entities}")
    
    results = []
    if not entities:
        print("No se proporcionaron entidades para filtrar. Considera devolver algunos productos populares o ninguno.")
        # results = product_database[:3] 
        results = [] 
    else:
        for product in product_database:
            match = True
            # Comprobar categoría
            entity_categoria = entities.get("categoria", "").lower()
            product_categoria = product.get("categoria", "").lower()
            if entity_categoria and entity_categoria not in product_categoria:
                match = False
            
            # Comprobar marca
            entity_marca = entities.get("marca", "").lower()
            product_marca = product.get("marca", "").lower()
            if entity_marca and entity_marca not in product_marca:
                match = False
            
            # Comprobar nombre_producto 
            entity_nombre = entities.get("nombre_producto", "").lower()
            product_nombre = product.get("nombre", "").lower()
            if entity_nombre and entity_nombre not in product_nombre: 
                match = False
            
            # Comprobar color 
            entity_color = entities.get("color", "").lower()
            product_colores = [str(c).lower() for c in product.get("colores", [])] # Asegurar que sean strings
            if entity_color and product_colores and entity_color not in product_colores:
                match = False
            
            # Comprobar talla 
            entity_talla = str(entities.get("talla", "")) 
            product_tallas = [str(t) for t in product.get("tallas_disponibles", [])] 
            if entity_talla and product_tallas and entity_talla not in product_tallas:
                match = False
            
            if match:
                results.append(product)

    if not results:
        print("No se encontraron productos que coincidan con las entidades.")
        current_call_log.append(f"CATALOG_QUERY: Entities='{json.dumps(entities)}', Result='No products found'")
    else:
        print(f"Productos encontrados: {len(results)}")
        summary_results = [{"id": p.get("id"), "nombre": p.get("nombre")} for p in results[:3]]
        current_call_log.append(f"CATALOG_QUERY: Entities='{json.dumps(entities)}', Found='{len(results)} items', ExampleResults='{json.dumps(summary_results)}'")

    return {
        "catalogQueryResult": results,
        "callLog": current_call_log
    }


### Definición del Nodo - Generar Respuesta

In [ ]:
def generate_response(state: AgentState):
    """
    Genera una respuesta en lenguaje natural basada en los resultados de la consulta al catálogo
    y la intención/entidades del usuario.
    """
    print("---NODO: Generando Respuesta---")
    user_input = state.get("userInput", "")
    intent = state.get("intent", "")
    entities = state.get("entities", {})
    catalog_results = state.get("catalogQueryResult", [])
    current_call_log = state.get("callLog", [])

    # Preparar el contexto para el LLM
    context_for_llm = f"La consulta original del usuario fue: '{user_input}'.\n"
    context_for_llm += f"La intención identificada fue: '{intent}'.\n"
    if entities:
        context_for_llm += f"Las entidades extraídas fueron: {json.dumps(entities)}.\n"

    if intent == "error_nlu":
        final_response_text = "Lo siento, tuve problemas para entender tu solicitud. ¿Podrías intentarlo de nuevo de otra manera?"
        current_call_log.append(f"RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}

    if intent in ["saludar", "despedirse"] and not catalog_results: 
        if intent == "saludar":
            final_response_text = "¡Hola! Soy tu asistente de compras. ¿En qué puedo ayudarte hoy?"
        elif intent == "despedirse":
            final_response_text = "¡Hasta luego! Que tengas un buen día."
        else: # Por si acaso
            final_response_text = "Entendido."
        current_call_log.append(f"RESPONSE_GEN: Intent='{intent}', Generated_Response='{final_response_text}'")
        return {"finalResponse": final_response_text, "callLog": current_call_log}


    if not catalog_results:
        if intent in ["buscar_producto", "pedir_recomendacion", "comparar_productos"]:
            context_for_llm += "No se encontraron productos que coincidan exactamente con tu búsqueda en el catálogo.\n"
            system_message_content = (
                "Eres un asistente de compras amigable y servicial. "
                "El usuario realizó una búsqueda pero no se encontraron productos. "
                "Informa al usuario de esto de manera amigable y quizás sugiere que intente una búsqueda diferente o más general. "
                "No inventes productos. Sé breve y directo."
            )
        else:
            context_for_llm += "No se requirió ni se obtuvo información del catálogo para esta consulta.\n"
            system_message_content = (
                "Eres un asistente de compras amigable y servicial. "
                "Responde al usuario de forma concisa basándote en la intención y el contexto proporcionado. "
                "Si la intención no está clara, pide una aclaración."
            )
    else:
        context_for_llm += "Se encontraron los siguientes productos en el catálogo que podrían ser relevantes:\n"
        for i, product in enumerate(catalog_results[:3]): 
            context_for_llm += f"  Producto {i+1}: {product.get('nombre', 'Nombre no disponible')} (Marca: {product.get('marca', 'N/A')}, Precio: ${product.get('precio', 'N/A')})\n"
            if product.get('descripcion'):
                 context_for_llm += f"    Descripción: {product.get('descripcion')}\n"
        if len(catalog_results) > 3:
            context_for_llm += f"  ... y {len(catalog_results) - 3} productos más.\n"
        
        system_message_content = (
            "Eres un asistente de compras amigable y servicial. "
            "Basándote en la consulta del usuario y los productos encontrados en el catálogo (proporcionados en el contexto), "
            "genera una respuesta útil y conversacional. Resume la información si es necesario. "
            "Si hay varios productos, puedes mencionar algunos y preguntar si el usuario desea más detalles sobre alguno en particular. "
            "No inventes productos ni características que no estén en la lista."
        )

    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessage(content=system_message_content),
        HumanMessage(content=context_for_llm + "\nPor favor, genera una respuesta adecuada para el usuario.")
    ])

    try:
        formatted_prompt = prompt_template.format_messages() 
        ai_response = llm.invoke(formatted_prompt)
        final_response_text = ai_response.content.strip()

        print(f"Respuesta generada por el LLM: {final_response_text}")
        current_call_log.append(f"RESPONSE_GEN: Intent='{intent}', Found_Items='{len(catalog_results)}', Generated_Response='{final_response_text[:200]}...'")

    except Exception as e:
        print(f"Error durante la generación de respuesta con el LLM: {e}")
        current_call_log.append(f"RESPONSE_GEN_ERROR: Error - {str(e)}")
        final_response_text = "Lo siento, estoy teniendo problemas para generar una respuesta en este momento. Intenta de nuevo más tarde."

    return {
        "finalResponse": final_response_text,
        "callLog": current_call_log
    }

### Ensamblaje del Grafo LangGraph

In [ ]:
workflow = StateGraph(AgentState)

workflow.add_node("nlu_parser", interpret_user_input)
workflow.add_node("catalog_tool", query_product_catalog)
workflow.add_node("response_generator", generate_response)

workflow.set_entry_point("nlu_parser")

workflow.add_edge("nlu_parser", "catalog_tool")

workflow.add_edge("catalog_tool", "response_generator")

workflow.add_edge("response_generator", END)


try:
    app = workflow.compile()
    print("Grafo LangGraph compilado exitosamente.")
except Exception as e:
    print(f"Error al compilar el grafo LangGraph: {e}")
    app = None

### Ejecutar el Agente con una Entrada de Ejemplo

In [ ]:
# Solo ejecutar si el grafo se compiló correctamente
if app:
    print("\n--- EJECUTANDO EL AGENTE ---")
    
    # Definir la entrada del usuario
    user_query = "Hola, estoy buscando un televisor SuperVision de 55 pulgadas, ¿tienes alguno?"
    # user_query = "Tienes zapatillas RunnerFlex rojas talla 42?"
    # user_query = "Qué tal?"
    # user_query = "Adiós"
    # user_query = "Busco un taladro percutor" # Ejemplo de algo que no está en el catálogo

    initial_state_input = {"userInput": user_query, "callLog": []} 

    print(f"Entrada del Usuario: \"{user_query}\"")
    
    # Invocar el agente con la entrada.
    
    try:
        final_state = app.invoke(initial_state_input)

        # Mostrar la respuesta final y el log de llamadas
        print("\n--- RESULTADO FINAL DEL AGENTE ---")
        print(f"Respuesta Final para el Usuario: {final_state.get('finalResponse', 'No se generó respuesta final.')}")
        
        print("\n--- LOG DE LLAMADAS (Rendimiento/Supervisión) ---")
        if final_state.get('callLog'):
            for log_entry in final_state['callLog']:
                print(log_entry)
        else:
            print("El log de llamadas está vacío.")
            
    except Exception as e:
        print(f"Error durante la ejecución del agente: {e}")
        print("Detalles del estado en el momento del error (si está disponible en la excepción):")
        
        # Algunas excepciones de LangGraph pueden llevar el estado parcial.
        if hasattr(e, 'state'):
             print(json.dumps(e.state, indent=2, ensure_ascii=False))

else:
    print("El agente no se pudo compilar. No se puede ejecutar.")
